# Session 22: TensorFlow (workshop)
### Presented by [Jeff Heaton, RGA](https://sites.wustl.edu/jeffheaton/)
For more examples like these, refer to my [class website](https://sites.wustl.edu/jeffheaton/t81-558/)

In [1]:
# Do you have TensorFlow installed?

import tensorflow

print(tensorflow.__version__)

ImportError: No module named 'tensorflow'